# Wine Quality Prediction Dashboard
This demo will consume a REST API fronting the latest trained model.

In [ ]:
from dashboard_utils import *

pn.extension()

# Build layout
pn.Row(
    pn.Column(
        fixed_acidity_float_slider,
        volatile_acidity_float_slider,
        citric_acid_float_slider,
        residual_sugar_float_slider,
        chlorides_float_slider,
        free_sulfur_dioxide_float_slider,
        total_sulfur_dioxide_float_slider,
        density_float_slider,
        pH_float_slider,
        sulphates_float_slider,
        alcohol_float_slider,
    ),
    pn.Column(pn.Row(submit_btn, clear_btn), results_df_pane),
).servable("Wine Quality Dashboard")